# Exploring kmer features from GISAID lineages

The features used in this notebook were generated by running
```sh
python featurize_gisaid.py  # takes about 5 minutes
```

In [ ]:
import torch
import matplotlib.pyplot as plt

In [ ]:
data = torch.load("results/gisaid.kmer_counts.pt")
print(list(data.keys()))
locals().update(data)
print(kmer_counts.shape)

Drop lineages with too few sequences

In [ ]:
min_lineages = 50
mask = (lineage_counts >= min_lineages)
lineage_counts = lineage_counts[mask]
kmer_counts = kmer_counts[mask]
lineages = [name for name, b in zip(lineage_ids, mask) if b]
print(kmer_counts.shape)

Quantize features

In [ ]:
lb, ub = 0.1, 0.8  # ad hoc thresholds
features = kmer_counts / lineage_counts[:, None]
features = (features > ub).float() - (raw_features < lb).float()  # in {-1, 0, 1}

Drop constant features

In [ ]:
mask = (features.max(0).values - features.min(0).values == 2)
features = features[:, mask]
print(features.shape)

Drop ambiguous features

In [ ]:
mask = features.abs().mean(0) > 0.6
features = features[:, mask]
print(features.shape)

In [ ]:
unique = sorted(set(map(tuple, features.T.long().tolist())))
print(len(unique))

In [ ]:
features = torch.tensor(list(map(list, unique)), dtype=torch.float).T.contiguous()
features.shape

In [ ]:
plt.imshow(features.numpy(), cmap='bwr')
plt.xticks(())
plt.yticks(())
plt.tight_layout()